# Process Amazon JHU Data

In [114]:
import re
import json
import pickle
import numpy as np

## Import data

In [43]:
path = "/Users/yuchen.zhang/Documents/Projects/domain-adaptation-nlp/data/amazon_jhu/"
types = ["books", "dvd", "electronics", "kitchen_&_housewares"]

In [95]:
def open_files(file_path):
    with open(file_path, "rb") as f:
        data = f.read().decode("utf-8", "ignore")
    return [x.split("<review_text>\n")[1] for x in data.split("\n</review_text>") if len(x.split("<review_text>\n"))==2]

In [99]:
data = {}
for t in types:
    data[t] = {"pos":open_files(path + "sorted_data/" + t + "/positive.review"), 
               "neg":open_files(path + "sorted_data/" + t + "/negative.review")}

In [104]:
with open(path + 'amazon_jhu4_text.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Embed Data

In [105]:
from transformers import RobertaTokenizer, RobertaModel
import torch

In [106]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [117]:
def tokenize_encode_bert_sentences_batch(tokenizer, model, input_sentences, output_path, layer_type="CLS"):
    output = np.zeros([len(input_sentences), 768])
    for i, x in enumerate(input_sentences):
        output[i] = tokenize_encode_bert_sentences_sample(tokenizer=tokenizer, model=model,
                                                          input_sentences=[x],
                                                          layer_type=layer_type)
    if output_path is not None:
        np.save(output_path, output)
    return output

def tokenize_encode_bert_sentences_sample(tokenizer, model, input_sentences, layer_type="CLS"):
    encoded_input = tokenizer(input_sentences, return_tensors='pt', truncation=True, padding=True)
    output = model(**encoded_input)[0]
    if layer_type == "CLS":
        output = output[:, 0, :].detach().numpy()
    elif layer_type == "first subtoken":
        output = output[:, 1, :].detach().numpy()
    else:
        output = output.detach().numpy()
    return output

In [120]:
embeds = {}
for t in data:
    embeds[t] = {"pos": tokenize_encode_bert_sentences_batch(tokenizer, model, data[t]["pos"], None, layer_type="CLS"),
                "neg": tokenize_encode_bert_sentences_batch(tokenizer, model, data[t]["neg"],None, layer_type="CLS")}

In [122]:
with open(path + 'amazon_jhu4_roberta.pickle', 'wb') as handle:
    pickle.dump(embeds, handle, protocol=pickle.HIGHEST_PROTOCOL)